<a href="https://colab.research.google.com/github/MocktaiLEngineer/100-days-of-GenAI/blob/main/TrigramLanguageModel_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as F

In [2]:
# Get the names.txt file
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

--2023-03-25 10:49:37--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: ‘names.txt’

names.txt           100%[===================>] 222.80K  --.-KB/s    in 0.01s   

2023-03-25 10:49:37 (22.2 MB/s) - ‘names.txt’ saved [228145/228145]



In [3]:
words = open('names.txt','r').read().splitlines()

In [4]:
TOKEN = '.'
vocab = [TOKEN] + sorted(list(set(''.join(words)))) 

In [5]:
n = len(vocab)

In [6]:
char_to_int = {char:i for i,char in enumerate(vocab)}
int_to_char = {i:char for char,i in char_to_int.items()}

In [7]:
# Prepare the dataset of bigrams (x,y)

X,Y = [],[]

for word in words:
    word = [TOKEN] + list(word) + [TOKEN]
    for ch1,ch2,ch3 in zip(word,word[1:],word[2:]):
        ix1 = char_to_int[ch1]
        ix2 = char_to_int[ch2]
        ix3 = char_to_int[ch3]
        X.append([ix1,ix2])
        Y.append(ix3)

X = torch.tensor(X)
Y = torch.tensor(Y)

print(X,Y)

# Initialising the network
g = torch.Generator().manual_seed(2147483647) # for reproducibility
W = torch.randn((n * 2,n), requires_grad = True, generator=g)
num = X.shape[0]

tensor([[ 0,  5],
        [ 5, 13],
        [13, 13],
        ...,
        [26, 25],
        [25, 26],
        [26, 24]]) tensor([13, 13,  1,  ..., 26, 24,  0])


In [12]:
for i in range(200):
    # Forward pass
    xenc = F.one_hot(X, num_classes = n).float()

    xenc_flattened = xenc.view(xenc.shape[0],-1)

    logits = xenc_flattened @ W #Log counts
    counts = logits.exp()
    probs = counts / torch.sum(counts, dim = 1, keepdim = True)
    loss = -probs[torch.arange(num), Y].log().mean() 

    # Backward pass
    W.grad = None
    loss.backward()

    # Update parameters
    W.data += -50 * W.grad

    print(f"Iteration:{i} | {loss=}")

Iteration:0 | loss=tensor(2.2399, grad_fn=<NegBackward0>)
Iteration:1 | loss=tensor(2.2399, grad_fn=<NegBackward0>)
Iteration:2 | loss=tensor(2.2399, grad_fn=<NegBackward0>)
Iteration:3 | loss=tensor(2.2399, grad_fn=<NegBackward0>)
Iteration:4 | loss=tensor(2.2399, grad_fn=<NegBackward0>)
Iteration:5 | loss=tensor(2.2399, grad_fn=<NegBackward0>)
Iteration:6 | loss=tensor(2.2399, grad_fn=<NegBackward0>)
Iteration:7 | loss=tensor(2.2399, grad_fn=<NegBackward0>)
Iteration:8 | loss=tensor(2.2399, grad_fn=<NegBackward0>)
Iteration:9 | loss=tensor(2.2398, grad_fn=<NegBackward0>)
Iteration:10 | loss=tensor(2.2398, grad_fn=<NegBackward0>)
Iteration:11 | loss=tensor(2.2398, grad_fn=<NegBackward0>)
Iteration:12 | loss=tensor(2.2398, grad_fn=<NegBackward0>)
Iteration:13 | loss=tensor(2.2398, grad_fn=<NegBackward0>)
Iteration:14 | loss=tensor(2.2398, grad_fn=<NegBackward0>)
Iteration:15 | loss=tensor(2.2398, grad_fn=<NegBackward0>)
Iteration:16 | loss=tensor(2.2398, grad_fn=<NegBackward0>)
Iterati

In [13]:
# Let's sample and generate
g = torch.Generator().manual_seed(2147483647) # for reproducibility

for i in range(10):
  name = []
  
  ix1 = 0
  ix2 = 0

  while True:
    xenc = F.one_hot(torch.tensor([ix1,ix2]), num_classes = n).float()
    
    xenc_flattened = xenc.view(1, -1)
    
    logits = xenc_flattened @ W #Log counts


    counts = logits.exp()
    probs = counts / torch.sum(counts, dim = 1, keepdim = True)
    ix2 = torch.multinomial(probs, num_samples = 1, replacement = True, generator = g).item()
    name.append(int_to_char[ix2])

    ix1 = ix2

    if ix2 == 0:
      break
  print(''.join(name))

nin.
avo.
militlynyleralumin.
arada.
nalyhizaridan.
an.
kegade.
ri.
adenemeshan.
adynar.
